# [선형회귀분석 실습 4: 실전]

## 모듈 불러오기

In [73]:
from IPython.display import display, HTML
import os
import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats as stats

import statsmodels.api as sm
import statsmodels.formula.api as smf

import pylab

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

#plt.rc('font', family='Malgun Gothic')

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true-y_pred)/y_true))*100

## 데이터 불러오기

### Toyota Corolla Data
### 중고차 가격(Price)를 예측하기 위한 예측모델 학습

In [74]:
data = pd.read_csv('C:/Users/ksk30/OneDrive/바탕 화면/머신러닝/AI-main/AI-main/ToyotaCorolla.csv')
data.head()

,Id,Model,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,Fuel_Type,HP,Met_Color,...,Central_Lock,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar
0,1,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13500,23,10,2002,46986,Diesel,90,1,...,1,1,1,0,0,0,1,0,0,0
1,2,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13750,23,10,2002,72937,Diesel,90,1,...,1,0,1,0,0,0,1,0,0,0
2,3,?TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13950,24,9,2002,41711,Diesel,90,1,...,0,0,1,0,0,0,1,0,0,0
3,4,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,14950,26,7,2002,48000,Diesel,90,0,...,0,0,1,0,0,0,1,0,0,0
4,5,TOYOTA Corolla 2.0 D4D HATCHB SOL 2/3-Doors,13750,30,3,2002,38500,Diesel,90,0,...,1,1,1,0,1,0,1,0,0,0


## 데이터 전처리하기
### 1. 예측에 필요하지 않은 변수 제거

In [75]:
data1 = data.drop(['Id', 'Model', 'Fuel_Type'], axis=1)
data1.head()

,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,HP,Met_Color,Automatic,cc,Doors,...,Central_Lock,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar
0,13500,23,10,2002,46986,90,1,0,2000,3,...,1,1,1,0,0,0,1,0,0,0
1,13750,23,10,2002,72937,90,1,0,2000,3,...,1,0,1,0,0,0,1,0,0,0
2,13950,24,9,2002,41711,90,1,0,2000,3,...,0,0,1,0,0,0,1,0,0,0
3,14950,26,7,2002,48000,90,0,0,2000,3,...,0,0,1,0,0,0,1,0,0,0
4,13750,30,3,2002,38500,90,0,0,2000,3,...,1,1,1,0,1,0,1,0,0,0


In [76]:
data1.describe()

,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,HP,Met_Color,Automatic,cc,Doors,...,Central_Lock,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar
count,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000,1436.00000,1436.000000,...,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000,1436.000000
mean,10730.824513,55.947075,5.548747,1999.625348,68533.259749,101.502089,0.674791,0.055710,1576.85585,4.033426,...,0.580084,0.561978,0.977716,0.146240,0.256964,0.300139,0.770195,0.204735,0.145543,0.277855
std,3626.964585,18.599988,3.354085,1.540722,37506.448872,14.981080,0.468616,0.229441,424.38677,0.952677,...,0.493717,0.496317,0.147657,0.353469,0.437111,0.458478,0.420854,0.403649,0.352770,0.448098
min,4350.000000,1.000000,1.000000,1998.000000,1.000000,69.000000,0.000000,0.000000,1300.00000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8450.000000,44.000000,3.000000,1998.000000,43000.000000,90.000000,0.000000,0.000000,1400.00000,3.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,9900.000000,61.000000,5.000000,1999.000000,63389.500000,110.000000,1.000000,0.000000,1600.00000,4.000000,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,11950.000000,70.000000,8.000000,2001.000000,87020.750000,110.000000,1.000000,0.000000,1600.00000,5.000000,...,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000
max,32500.000000,80.000000,12.000000,2004.000000,243000.000000,192.000000,1.000000,1.000000,16000.00000,5.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [77]:
#sns.pairplot(data1)

In [78]:
Y = data1['Price']
X = data1.drop('Price', axis=1)

### 2. 학습 데이터와 테스트 데이터로 분리

In [79]:
train_data, test_data = train_test_split(data1, test_size=0.3, random_state=55)

In [80]:
train_data.shape

(1005, 34)

In [81]:
train_data.head()

,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,HP,Met_Color,Automatic,cc,Doors,...,Central_Lock,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar
1236,7450,78,3,1998,82675,86,0,0,1300,3,...,0,0,1,0,0,1,1,0,0,0
130,16250,20,1,2003,32627,97,1,0,1400,5,...,1,1,1,0,0,1,1,0,0,0
1356,8400,77,4,1998,60000,110,1,0,1600,3,...,1,1,1,0,0,1,1,0,0,0
770,7950,64,5,1999,78356,86,1,0,1300,3,...,0,0,1,0,0,1,1,0,0,1
1332,8495,77,4,1998,64280,110,0,0,1600,4,...,0,0,1,0,0,0,0,1,0,0


In [82]:
Y = data1['Price']
X = data1.drop('Price', axis=1)
X_train_data, X_test_data, Y_train_data, Y_test_data  = train_test_split(X, Y, test_size=0.3, random_state=55)

In [83]:
X_train_data.head()

,Age_08_04,Mfg_Month,Mfg_Year,KM,HP,Met_Color,Automatic,cc,Doors,Cylinders,...,Central_Lock,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar
1236,78,3,1998,82675,86,0,0,1300,3,4,...,0,0,1,0,0,1,1,0,0,0
130,20,1,2003,32627,97,1,0,1400,5,4,...,1,1,1,0,0,1,1,0,0,0
1356,77,4,1998,60000,110,1,0,1600,3,4,...,1,1,1,0,0,1,1,0,0,0
770,64,5,1999,78356,86,1,0,1300,3,4,...,0,0,1,0,0,1,1,0,0,1
1332,77,4,1998,64280,110,0,0,1600,4,4,...,0,0,1,0,0,0,0,1,0,0


## 모델링

### 학습 데이터를 이용하여 선형회귀모델 학습

In [84]:
lm = sm.OLS(train_data['Price'], train_data.drop('Price', axis=1))
lm_trained = lm.fit()

### 모델 학습 결과 (Summary) 확인

In [85]:
display(lm_trained.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.920
Model:                            OLS   Adj. R-squared:                  0.918
Method:                 Least Squares   F-statistic:                     375.5
Date:                Fri, 14 May 2021   Prob (F-statistic):               0.00
Time:                        11:15:58   Log-Likelihood:                -8436.6
No. Observations:                1005   AIC:                         1.694e+04
Df Residuals:                     974   BIC:                         1.709e+04
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Age_08_04         -114.9816      4.017    -28.624      0.000    -122.865    -107.099
Mfg_Month          -93.2776     10.543     -8.848      0.000    -113.966     -72.589
Mfg_Year            -2.1563      0.848     -2.542      0.011      -3.821      -0.492
KM                  -0.0191      0.001    -15.524      0.000      -0.021      -0.017
HP                  18.5371      2.843      6.519      0.000      12.957      24.117
Met_Color         -103.0730     79.112     -1.303      0.193    -258.322      52.176
Automatic          195.8010    148.789      1.316      0.188     -96.183     487.785
cc                  -0.1373      0.076     -1.803      0.072      -0.287       0.012
Doors              -15.6624     41.785     -0.375      0.708     -97.662      66.337
Cylinders           -0.0389      0.002    -17.095      0.000      -0.043      -0.034
Gears              204.1533    187.482      1.089      0.276    -163.762     572.069
Quarterly_Tax        1.3659      1.478      0.924      0.356      -1.535       4.267
Weight              18.8386      1.431     13.165      0.000      16.030      21.647
Mfr_Guarantee      312.9440     77.283      4.049      0.000     161.283     464.605
BOVAG_Guarantee    679.1090    131.481      5.165      0.000     421.090     937.128
Guarantee_Period    76.7554     13.590      5.648      0.000      50.086     103.425
ABS               -414.1815    131.349     -3.153      0.002    -671.942    -156.421
Airbag_1          -325.9419    249.517     -1.306      0.192    -815.595     163.711
Airbag_2           161.7414    135.909      1.190      0.234    -104.967     428.450
Airco               85.6836     92.377      0.928      0.354     -95.598     266.965
Automatic_airco   2148.0897    193.528     11.100      0.000    1768.310    2527.870
Boardcomputer     -375.3768    123.072     -3.050      0.002    -616.893    -133.861
CD_Player          277.0970    104.093      2.662      0.008      72.825     481.369
Central_Lock      -246.8519    143.376     -1.722      0.085    -528.214      34.510
Powered_Windows    663.8474    143.354      4.631      0.000     382.530     945.165
Power_Steering    -397.8024    288.134     -1.381      0.168    -963.237     167.632
Radio              -47.4025     53.746     -0.882      0.378    -152.875      58.070
Mistlamps          -45.0181    114.003     -0.395      0.693    -268.738     178.702
Sport_Model        210.4336     91.325      2.304      0.021      31.217     389.650
Backseat_Divider   -85.2049    128.238     -0.664      0.507    -336.860     166.450
Metallic_Rim       236.9277     97.772      2.423      0.016      45.059     428.797
Radio_cassette     -47.4025     53.746     -0.882      0.378    -152.875      58.070
Tow_Bar           -100.1751     83.535     -1.199      0.231    -264.104      63.753
=======================

In [86]:
# data2 = data.drop(['Id', 'Model', 'Fuel_Type','Mfg_Year','Met_Color','Automatic','cc','Doors','Gears','Quarterly_Tax','Quarterly_Tax','ABS','Airbag_1','Airbag_2','Airco','Boardcomputer','CD_Player','Central_Lock','Power_Steering','Radio','Mistlamps','Sport_Model','Backseat_Divider','Metallic_Rim','Radio_cassette','Tow_Bar'], axis=1)
# data2.head()

In [89]:
threshold = 0.05
delete_index = lm_trained.pvalues.index[lm_trained.pvalues >= threshold]

In [64]:
train_data, test_data = train_test_split(data2, test_size=0.3, random_state=55)
lm = sm.OLS(train_data['Price'], train_data.drop('Price', axis=1))
lm_trained = lm.fit()
display(lm_trained.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.915
Model:                            OLS   Adj. R-squared:                  0.914
Method:                 Least Squares   F-statistic:                     1066.
Date:                Fri, 14 May 2021   Prob (F-statistic):               0.00
Time:                        11:13:09   Log-Likelihood:                -8471.5
No. Observations:                1005   AIC:                         1.697e+04
Df Residuals:                     994   BIC:                         1.702e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Age_08_04         -107.7284      2.750    -39.181      0.000    -113.124    -102.333
Mfg_Month          -86.9827     10.601     -8.205      0.000    -107.786     -66.180
KM                  -0.0196      0.001    -16.306      0.000      -0.022      -0.017
HP                  17.5918      2.529      6.957      0.000      12.630      22.554
Cylinders        -1378.8321    267.048     -5.163      0.000   -1902.874    -854.790
Weight              19.5261      0.910     21.468      0.000      17.741      21.311
Mfr_Guarantee      300.8841     76.112      3.953      0.000     151.526     450.242
BOVAG_Guarantee    652.3062    123.456      5.284      0.000     410.042     894.571
Guarantee_Period    78.8404     12.322      6.398      0.000      54.659     103.021
Automatic_airco   2344.6834    179.076     13.093      0.000    1993.273    2696.094
Powered_Windows    482.3003     76.745      6.284      0.000     331.700     632.901
==============================================================================
Omnibus:                       46.881   Durbin-Watson:                   2.073
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              149.209
Skew:                          -0.021   Prob(JB):                     3.98e-33
Kurtosis:                       4.887   Cond. No.                     6.13e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.13e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [90]:
train_data.drop(['Price'], axis=1)

,Age_08_04,Mfg_Month,Mfg_Year,KM,HP,Met_Color,Automatic,cc,Doors,Cylinders,...,Central_Lock,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar
1236,78,3,1998,82675,86,0,0,1300,3,4,...,0,0,1,0,0,1,1,0,0,0
130,20,1,2003,32627,97,1,0,1400,5,4,...,1,1,1,0,0,1,1,0,0,0
1356,77,4,1998,60000,110,1,0,1600,3,4,...,1,1,1,0,0,1,1,0,0,0
770,64,5,1999,78356,86,1,0,1300,3,4,...,0,0,1,0,0,1,1,0,0,1
1332,77,4,1998,64280,110,0,0,1600,4,4,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,54,3,2000,46230,110,1,0,1600,3,4,...,1,1,1,0,1,0,1,1,0,0
968,63,6,1999,48738,110,0,0,1600,5,4,...,1,1,1,0,0,0,0,0,0,1
295,38,7,2001,41754,110,0,0,1600,3,4,...,0,0,1,0,0,0,1,0,0,0
666,68,1,1999,109150,110,0,0,1600,5,4,...,1,1,1,0,1,0,1,0,0,0


In [91]:
train_data, test_data = train_test_split(data1, test_size=0.3, random_state=55)
lm2 = LR()
lm_trained = lm2.fit(train_data.drop(['Price'], axis=1), train_data['Price'])

## 예측성능 평가

### 1. 학습 데이터셋에 대한 예측성능 평가
MSE, RMSE, MAE, MAPE 계산하여 성능 확인

In [92]:
test_pred = lm_trained.predict(test_data.drop('Price', axis=1))

In [93]:
test_mse = mean_squared_error(test_data['Price'], test_pred)
test_rmse = np.sqrt(mean_squared_error(test_data['Price'], test_pred))
test_mae = mean_absolute_error(test_data['Price'], test_pred)
test_mape = mean_absolute_percentage_error(test_data['Price'], test_pred)

In [94]:
print('Testing MSE: {:.3f}'.format(test_mse))
print('Testing RMSE: {:.3f}'.format(test_rmse))
print('Testing MAE: {:.3f}'.format(test_mae))
print('Testing MAPE: {:.3f}'.format(test_mape))

Testing MSE: 1788162.286
Testing RMSE: 1337.222
Testing MAE: 878.352
Testing MAPE: 9.200


### 2. 테스트 데이터셋에 대한 예측성능 평가
MSE, RMSE, MAE, MAPE 계산하여 성능 확인

## 변수 선택

### 1-1. 선형회귀모델에서 유의하지 않은 변수가 있다면 제거

### 1-2. 변수 제거 후 모델 학습 결과 확인

### 1-3. 변수 제거 후 모델 예측성능 (학습/테스트) 확인

### 2-1. 상관계수가 높은 설명변수들이 있다면 하나의 변수만 선택

### 2-2. 변수 제거 후 모델 학습 결과 확인

### 2-3. 변수 제거 후 모델 예측성능 (학습/테스트) 확인

## 변수 변환

### 반응변수($\mathbf{y}$)에 로그 / 제곱근 등의 변환 적용하고 모델 성능 확인